In [14]:
import mysql.connector
import csv
import sys
from datetime import datetime

# Database connection details
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'HunnyS@1511',
    'database': 'patient_readmission'
}

# File path
file_path = r"C:\Users\Hp\Desktop\IITK\patient readmission project\medications2.csv"

def convert_date(date_string):
    try:
        return datetime.strptime(date_string, '%d-%m-%Y').strftime('%Y-%m-%d')
    except ValueError:
        return None

def delete_and_load_data(cursor, conn):
    try:
        # Delete all data from medications table
        cursor.execute("DELETE FROM medications")
        print("All existing data deleted from medications table.")

        # Read and insert data from CSV
        with open(file_path, 'r') as file:
            csv_reader = csv.reader(file)
            headers = next(csv_reader)  # Get column names
            
            # Prepare the INSERT query
            columns = ', '.join(headers)
            placeholders = ', '.join(['%s'] * len(headers))
            query = f"INSERT INTO medications ({columns}) VALUES ({placeholders})"
            
            # Insert data row by row
            for i, row in enumerate(csv_reader, start=1):
                try:
                    # Convert dates
                    row[3] = convert_date(row[3])  # start_date
                    row[4] = convert_date(row[4])  # end_date
                    
                    cursor.execute(query, row)
                    if i % 1000 == 0:  # Commit every 1000 rows
                        conn.commit()
                        print(f"{i} rows inserted.")
                except mysql.connector.Error as err:
                    print(f"Error at row {i}: {err}")
                    print(f"Problematic row: {row}")
                    conn.rollback()
            
            # Final commit
            conn.commit()
            print(f"All data inserted. Total rows: {i}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        conn.rollback()

def main():
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        delete_and_load_data(cursor, conn)

        print("Data reload completed successfully.")

    except mysql.connector.Error as err:
        print(f"Database connection error: {err}")
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    main()

All existing data deleted from medications table.
1000 rows inserted.
2000 rows inserted.
3000 rows inserted.
4000 rows inserted.
5000 rows inserted.
6000 rows inserted.
7000 rows inserted.
8000 rows inserted.
9000 rows inserted.
10000 rows inserted.
All data inserted. Total rows: 10000
Data reload completed successfully.
MySQL connection is closed
